# Lab 13: Photometry
Based on the AAS2016 photuntls tutorial.
- Documentation: https://photutils.readthedocs.io/en/stable/


## What is Aperture Photometry?
Aperture photometry is one method we use to convert astronomical images into measurements of brightness for individual stars. In this particular case we will use circular apertures for stars, but the same basic principals work for ellipse etc. around galaxies and other diffuse objects. The fundamental idea is to sum up the number of counts in the aperture to determine how many photons where observed. We also need to subtract off background photons.

In [ ]:
# initial imports
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import scipy.stats
import astropy.units as u

# change some default plotting parameters
import matplotlib as mpl
mpl.rcParams['image.origin'] = 'lower'
mpl.rcParams['image.interpolation'] = 'nearest'
mpl.rcParams['image.cmap'] = 'Greys_r'

# run the %matplotlib magic command to enable inline plotting
# in the current Notebook
%matplotlib inline

## Load our image

In [ ]:
#Read in our image
hdulist = fits.open('data/aa_aql0007.fits')
hdulist.info()
prihdr = hdulist[0].header
image_data = hdulist[0].data.astype(np.float) *u.adu #Ensure a good data type and units
image_data = image_data[0:600,0:600] #Shrink our image
print(np.median(image_data))

In [ ]:
#Get Useful header info
read_noise = np.float(prihdr['rdnoise']) *u.electron
gain = np.float(prihdr['gain']) * u.electron / u.adu
exptime = np.float(prihdr['exptime']) * u.s
print(read_noise)
print(gain)
print(exptime)

In [ ]:
#We also need to know the dark current.
dark = fits.open('data/Dark.fits')
dark_hdr = dark[0].header
dark_data = dark[0].data *u.adu * gain
dark_current = np.median(dark_data/(dark_hdr['exptime']*u.s))
print(dark_current)
dark.close() #Save some memory

## Gain
The image needs to be in electrons per second

In [ ]:
image_data = image_data * gain

In [ ]:
#View our image
from astropy.visualization import ZScaleInterval
interval = ZScaleInterval()
(imin,imax) = interval.get_limits(image_data)
plt.imshow(image_data, vmin=imin,vmax=imax)
plt.colorbar()

## Defining Apertures
For stellar photometry we will use circular apertures. Let's take an aperture radius 15 pixels around a star at (439.795,502.256) python space. Apertures are defined using (x,y) instead of (y,x) and start counting from (0,0). Apertures can be defined in several different ways because we are going from a continuous space to a discrete one. ![](data/photutils_aperture_methods.png)

In [ ]:
from photutils import CircularAperture, aperture_photometry
# define the aperture
position = (502.256,439.795)
radius = 15
aperture = CircularAperture(position, r=radius)

In [ ]:
# center method
phot = aperture_photometry(image_data, aperture,  method='center')
phot

In [ ]:
# subpixel method, subpixels=5 (same as Source Extractor (SExtractor))
phot = aperture_photometry(image_data, aperture,  method='subpixel', subpixels=5)
phot

In [ ]:
# perform the photometry; the default method is 'exact'
phot = aperture_photometry(image_data, aperture)
phot

## Multiple Positions

In [ ]:
positions = [(502.256,439.795), (354.291,363.823)]
radius = 15.
apertures = CircularAperture(positions, r=radius)
phot = aperture_photometry(image_data, apertures)
phot

In [ ]:
from astropy.visualization import ZScaleInterval
interval = ZScaleInterval()
(imin,imax) = interval.get_limits(image_data)
plt.imshow(image_data, vmin=imin,vmax=imax)
plt.colorbar()
apertures.plot(color='blue')

## Measure Sky Background
We want to remove the sky background from our pixels. Note the sky background does contribute to the noise

In [ ]:
from photutils import CircularAnnulus
bkg_apertures = CircularAnnulus(positions, r_in=30., r_out=35.)

In [ ]:
from astropy.visualization import ZScaleInterval
interval = ZScaleInterval()
(imin,imax) = interval.get_limits(image_data)
plt.imshow(image_data, vmin=imin,vmax=imax)
plt.colorbar()
apertures.plot(color='blue')
bkg_apertures.plot(color='cyan', hatch='//', alpha=0.8)

In [ ]:
# measure the aperture sum for the star
phot = aperture_photometry(image_data, apertures)

In [ ]:
#It turns out that the best measurement of the backgroud for photometry is not the mean, but rather the mode
def getBackground(image_data, apertures):
    bkgs = list()
    for m in apertures.to_mask('center'):
        inc_pix = m.cutout(image_data)[m.cutout(image_data) *m.data >0]
        mode_bak = scipy.stats.mode(inc_pix)
        bkgs.append(float(mode_bak[0]))
    return np.array(bkgs)*u.electron

In [ ]:
#Get the mode of the background for each stars
bkg_modes = getBackground(image_data,bkg_apertures)
print(bkg_modes)

In [ ]:
# now calculate the total background in the circular aperture
print(apertures.area)
bkg_sums = bkg_modes * apertures.area

phot['bkg_sum'] = bkg_sums
phot

In [ ]:
# subtract the background
flux_bkgsub = phot['aperture_sum'] - bkg_sums

phot['aperture_sum_bkgsub'] = flux_bkgsub
phot

## Photometric Error
The Error in our photometry comes from two sources Poisson noise noise in our science and calibration frames and readnoise. We can use a variation of Equation 9.72 in Chromey (2016): assuming $a_z \approx a_d \approx 1$

$\sigma^2_N = N_{*} + n_{pix}a_b(<b> + t\dot{d} + \rho^2$

$a_b \approx 1 + \frac{n_pix}{p_b}$

In [ ]:
a_b = 1 + (apertures.area/bkg_apertures.area)
Nvariance = phot['aperture_sum_bkgsub']*u.electron + apertures.area*a_b*(
    bkg_modes*u.electron+exptime*dark_current*u.electron + read_noise**2)
print(Nvariance)
Nsigma = np.sqrt(Nvariance)
print(Nsigma)

In [ ]:
# input the data units
phot['aperture_sum_bkgsub_err'] = Nsigma
phot

## Convert to Magnitudes
The final step is to convert to magnitudes. Note that logarithms have to be unitless, so I take the value part of my quantities.

In [ ]:
zpt = 25.
mag = zpt - 2.5*np.log10(phot['aperture_sum_bkgsub'].value) + 2.5*np.log10(exptime.value)
merr = 1.0857 *phot['aperture_sum_bkgsub_err']/phot['aperture_sum_bkgsub']
phot['Mag'] = mag * u.mag
phot['Mag_err'] = merr * u.mag
phot

## Lab 13: Now it is your turn
Please answer the following questions, then print them off and turn them in. You don't need to print the whole notebook. Only print the pages starting from here.

Name:

**Q1: Find the flux from the source in electrons for the three bright stars in the lower left corner of the image.**

**Q2: Find the error in the flux from the source in electrons for the three stars in Q1.**

**Q3: Find the magnitudes and errors for the three stars in Q1.**